In [12]:
import igraph 
from igraph import* 
import csv
from geopy.distance import geodesic 
import time;
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

#-.-.-.-.-.-.-OBTENER ESTACIONES (vertices del grafo) DE CSV-.-.-.-.-.-.-

In [13]:
heuristica={}

In [14]:
with open('estaciones.csv', mode='r') as file:
    # Crea un lector CSV
    reader = csv.reader(file)
    
    # Salta la primera fila que contiene los encabezados
    next(reader)
    
    # Inicializa una lista para almacenar los valores de la columna de estaciones
    nombre = []
    coordenada = {}
    # Itera a traves de las filas del archivo CSV
    for row in reader:
        # Agrega el valor de la columna "Estacion" a la lista
        nombre.append(row[1])
        x = (float)(row[3]) # Coordenadas X
        y = (float)(row[4]) # Coordenadas Y
        coordenada.update({row[1]:[x,y]})

#-.-.-.-.-.-.-OBTENER ARISTAS Y PESOS(conexiones entre estaciones) DE CSV-.-.-.-.-.-.-

In [15]:
with open('aristas.csv', mode='r') as file:
    # Crea un lector CSV
    reader = csv.reader(file)
    
    # Salta la primera fila que contiene los encabezados
    next(reader)
    
    # Inicializa una lista para almacenar las aristas y pesos
    aristas = []
    pesos = []

    # Itera a traves de las filas del archivo CSV
    for i, row in enumerate(reader):
        for j, valor in enumerate(row[1:]):  # Empezamos desde la columna 1 para omitir la primera columna
            if( i<= j): # Si no se ha llegado aun a la diagonal principal (Queremos evitar aristas repetidas)
                if valor != "0":
                    pesos.append(float(valor))
                    aristas.append((i, j))

In [16]:
def calculoHeuristica(destino):
    destinoX = coordenada.get(destino)[0]
    destinoY = coordenada.get(destino)[1]
    for origen in coordenada:
        origenX = coordenada.get(origen)[0]
        origenY = coordenada.get(origen)[1]
        distanciaLineaRecta = geodesic((destinoX,destinoY),(origenX,origenY)).km
        heuristica.update({origen:distanciaLineaRecta})

#-.-.-.-.-.-.-PREPARAR GRAFO-.-.-.-.-.-.-

In [17]:
n_vertices = 40
print(aristas)
grafo = igraph.Graph(n_vertices, aristas)   # Creamos grafo
grafo["title"] = "Metro de Lyon" # Titulo del grafo
grafo.vs["name"] = nombre   # Nombre de los nodos
grafo.es["pesos"] = pesos   # Pesos de las arista
grafo.vs["color"] = ['red','red','red','red','red','red','red', 'red','red','red','red','red','red','red','red', 'red', 'red','red','red','red','red','red','red', 'red', 'red','red','red','red','red','red','red', 'red', 'red','red','red','red','red','red','red', 'red', 'red','red','red','red','red','red','red', 'red']
grafo.vs["label"] = nombre
grafo.vs["size"] = 70
grafo.vs["label_dist"] = 1.5
grafo.vs["label_size"] = 20

#print(grafo.vs["name"])
#print(grafo)

[(0, 1), (1, 2), (2, 3), (2, 30), (2, 31), (3, 4), (4, 5), (4, 26), (5, 6), (6, 7), (7, 8), (7, 14), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (14, 15), (15, 16), (16, 17), (17, 18), (17, 31), (17, 32), (18, 19), (19, 20), (20, 21), (21, 22), (23, 24), (24, 25), (25, 26), (27, 28), (28, 29), (29, 30), (32, 33), (33, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39)]


#-.-.-.-.-.-.-ALGORITMO ASTAR-.-.-.-.-.-.-

In [18]:
class Nodo:     
    def __init__(self, estacion, g = 0, h = 0):  
        self.estacion = estacion     #el propio nodo
        self.padre = None          #nodo padre
        self.g = g                 #valor g
        self.h = h                 #valor h
        self.f = 0                 #valor f
  

In [19]:
algoritmoAStar("Foch","Cuire")

Foch
2.2580954554690167
0
0
----------


NameError: name 'explorarVecino' is not defined

In [ ]:
def algoritmoAStar(Inicio,Destino):
   # inicialización de variable
   estacioInicio = grafo.vs.find(Inicio)
   estacionDestino = grafo.vs.find(Destino)
   coste = 0
   calculoHeuristica(estacionDestino["name"])
   listaNodoAbierto = {}
   listaNodoCerrado = {}
   ruta = []
   nodoInicio = Nodo(estacioInicio)
   nodoDestino = Nodo(estacionDestino)
   nodoActual = nodoInicio
   nodoActual.f = heuristica.get(Inicio)
   listaNodoAbierto.update({estacioInicio["name"]:nodoActual})
   
   while(listaNodoAbierto):
   # seleccionar el nodo con f menor y elimian de la lista abierto añadir en la lista cerrado
      print(nodoActual.estacion["name"])
      print(nodoActual.f)
      print(nodoActual.g)
      print(nodoActual.h)
      print("----------")
      if nodoActual.estacion == nodoDestino.estacion:
         coste = nodoActual.f
         ruta.append(nodoActual.estacion["name"])
         nodo = nodoActual.padre
         while(nodo):
            ruta.append(nodo.estacion["name"])
            nodo = nodo.padre
         listaNodoAbierto = {}
      else:
         listaNodoCerrado.update({nodoActual.estacion["name"]:nodoActual})
         del listaNodoAbierto[nodoActual.estacion["name"]]
         explorarVecino(grafo,heuristica,listaNodoAbierto,listaNodoCerrado,nodoActual)
         nodoActual = getNodoMinimo(listaNodoAbierto)

   for estacion in reversed(ruta):
      print(estacion)
   print("coste de distancia es {:.1f}km".format(coste))

In [21]:
#metodo que devuelve un nodo con f menor en la lista de nodo abierto 
def getNodoMinimo(listaNodoAbierto):
    menor = 11111111111
    for estacion in listaNodoAbierto:
        nodo = listaNodoAbierto.get(estacion)  
        if nodo.f < menor:  
            nodoMenor = nodo
            menor =  nodo.f 
    return nodoMenor

In [22]:
#exploral todos los nodos vecino de nodo actual y calcuar la f 
def explorarVecino(grafo:igraph,heuristica,listaNodoAbierto,listaNodoCerrado,nodoActual):
    verticeVecino = nodoActual.estacion.neighbors()
    for estacion in verticeVecino:
      
        if estacion["name"] in listaNodoCerrado:
            continue
        nodo = Nodo(estacion)
        indiceArista = grafo.get_eid(nodoActual.estacion,estacion)
        peso = grafo.es[indiceArista]["pesos"]
        g = peso + nodoActual.g
        h = heuristica.get(estacion["name"])
        f = g+h
        nodo.g = g
        nodo.h = h
        nodo.f = f
        nodo.padre = nodoActual

        if estacion in listaNodoAbierto:
            nodoAux = listaNodoAbierto.get(estacion["name"])
            if nodo.f < nodoAux.f:
                listaNodoAbierto.update({estacion["name"]:nodo})
        else:
             listaNodoAbierto.update({estacion["name"]:nodo})

In [ ]:
#algoritmoAStar()
igraph.plot(grafo, bbox=(0, 0, 2500, 2500))

In [24]:
app = tk.Tk()
origen = tk.StringVar(app)
destino = tk.StringVar(app) 

def boton():
    inicio = origen.get()
    final = destino.get()
    algoritmoAStar(inicio, final)

app.geometry("1920x1080")
app.configure(background="black")
tk.Wm.wm_title(app,"A*")
tk.Button(app,
          text="Click Me!",
          font=("Courier", 14),
          bg="#00a8e8",
          fg="blue",
          command=boton,
          height=2,
          width=15 
).pack()

figura = Figure(figsize=(8, 8))

# Obtener el subplot de la figura
subplot = figura.add_subplot(111)

# Visualizar el grafo en el subplot
igraph.plot(grafo, target=subplot, bbox=(0, 0, 200, 200))

# Crear el lienzo de Matplotlib para integrarlo en la interfaz de Tkinter
canvas = FigureCanvasTkAgg(figura, master=app)
canvas.draw()

# Colocar el lienzo en la ventana principal
canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

tk.Label(app, text="ORIGEN:").pack(side=tk.LEFT)

tk.Entry(app,
    fg="blue",
    bg="white",
    justify="center",
    textvariable=origen
).pack(side=tk.LEFT)

tk.Label(app, text="DESTINO:").pack(side=tk.LEFT)

tk.Entry(app,
    fg="blue",
    bg="white",
    justify="center",
    textvariable=destino
).pack(side=tk.LEFT)


app.mainloop()


2023-12-06 12:39:53.399 Python[33967:1380655] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


Foch
2.2580954554690167
0
0
----------
Hotel de Ville Louis Pradel
2.7775542285448314
0.7
2.077554228544831
----------
Croix-Paquet
2.788845893090006
1.1
1.688845893090006
----------
Croix-Rousse
2.8960223922448964
1.6
1.2960223922448966
----------
Henon
3.2966199633026476
2.3
0.9966199633026478
----------
Cuire
3.0999999999999996
3.0999999999999996
0.0
----------
Foch
Hotel de Ville Louis Pradel
Croix-Paquet
Croix-Rousse
Henon
Cuire
coste de distancia es 3.1km
